In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv


In [2]:

import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


In [3]:

os.environ['TORCH'] = torch.__version__

# !pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch_geometric pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-${TORCH}.html


Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 70.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 19.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 90.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 18.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.0 MB/s eta 0:00:00ta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00a 0:00:01


In [4]:
from  torch_geometric.nn import GCNConv,SAGEConv,GATConv,HeteroConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import Data,HeteroData

In [5]:
df_rating=pd.read_csv("/kaggle/input/movielens-20m-dataset/rating.csv")
df_movie=pd.read_csv("/kaggle/input/movielens-20m-dataset/movie.csv")

In [6]:
df_movie.head(-1)

df_movie['movieId'].unique

df_movie['movieId'].

df_rating['userId'].shape

df_rating['userId'].unique


df_rating['movieId'].nunique

df_rating.head()

df_rating.isnull().sum()

df_rating.duplicated().sum()



SyntaxError: invalid syntax (761601829.py, line 5)

In [7]:
df_rating['userId']=df_rating['userId'].astype('category').cat.codes
df_rating['movieId']=df_rating['movieId'].astype('category').cat.codes

num_users=int(df_rating['userId'].nunique())
num_items=int(df_rating['movieId'].nunique())

print("NUM users:",num_users)
print("num items: ",num_items)

NUM users: 138493
num items:  26744


In [8]:
device="cuda" if torch.cuda.is_available() else "cpu"

In [9]:
def create_graph(df_rating, df_movie):
    # (a) Keep only movies that appear in the ratings
    rated_ids = df_rating['movieId'].unique()
    df_movie = df_movie[df_movie['movieId'].isin(rated_ids)].copy()
    
    # (b) Category‐encode userId and movieId (only rated movies)
    df_rating['user_code']  = pd.Categorical(df_rating['userId']).codes
    df_rating['movie_code'] = pd.Categorical(df_rating['movieId'], 
                                             categories=rated_ids).codes
    num_users  = df_rating['user_code'].nunique()
    num_movies = df_rating['movie_code'].nunique()
    
    # (c) Build genre‐one-hot matrix for exactly these movie_codes
    # Map original movieId → code
    movie_to_code = dict(zip(rated_ids, range(num_movies)))
    df_movie['movie_code'] = df_movie['movieId'].map(movie_to_code)
    
    # Drop duplicates so each movie_code appears once with its genres
    gm = ( df_movie[['movie_code','genres']]
           .drop_duplicates('movie_code')
           .set_index('movie_code')['genres']
           .str.get_dummies(sep='|') )
    # Reindex 0…num_movies-1, fill missing with all zeros
    gm = gm.reindex(range(num_movies), fill_value=0)
    movie_feats = torch.tensor(gm.values, dtype=torch.float)   # [num_movies, num_genres]
    
    # (d) Dummy user features (or you can plug in real ones)
    user_feats = torch.arange(num_users, dtype=torch.float).view(-1,1)
    
    # (e) Build edge_index and edge_attr from df_rating
    edge_index = torch.tensor([
        df_rating['user_code'].values,
        df_rating['movie_code'].values
    ], dtype=torch.long)
    edge_attr  = torch.tensor(df_rating['rating'].values, dtype=torch.float)
    
    # (f) Assemble HeteroData
    graph = HeteroData()
    graph['user'].x = user_feats
    graph['movie'].x = movie_feats
    graph['user','rates','movie'].edge_index = edge_index
    graph['user','rates','movie'].edge_attr  = edge_attr
    graph['movie','rated_by','user'].edge_index = edge_index[[1,0]]
    
    return graph



In [10]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_rating, test_size=0.2, random_state=42)

In [11]:
train_graph=create_graph(train_data,df_movie).to(device)
test_graph=create_graph(test_data,df_movie).to(device)

/tmp/ipykernel_35/1611852729.py:31: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  edge_index = torch.tensor([


In [12]:
train_graph

HeteroData(
  user={ x=[138493, 1] },
  movie={ x=[25827, 19] },
  (user, rates, movie)={
    edge_index=[2, 16000210],
    edge_attr=[16000210],
  },
  (movie, rated_by, user)={ edge_index=[2, 16000210] }
)

In [13]:

print("total user nodes",train_graph['user'].x.shape[0])
print("total movies :", train_graph['movie'].x.shape[0])
print(" - user:", train_graph['user', 'rates', 'movie'].edge_index[0].max().item())
print(" - movie:", train_graph['user', 'rates', 'movie'].edge_index[1].max().item())
print(" - movie :", train_graph['movie', 'rated_by', 'user'].edge_index[0].max().item())
print(" - user:", train_graph['movie', 'rated_by', 'user'].edge_index[1].max().item())

total user nodes 138493
total movies : 25827
 - user: 138492
 - movie: 25826
 - movie : 25826
 - user: 138492


In [14]:
from torch_geometric.nn import MessagePassing
import torch.nn.functional as F
from torch_scatter import scatter_softmax
from torch.nn import Linear, ModuleDict, BatchNorm1d, Dropout


class AttAwareConv(MessagePassing):
    def __init__(self):
        super().__init__(aggr='add')
        self.att=torch.nn.Linear(1,1)

    def forward(self,x_user,x_movie,edge_index,edge_attr):

        if edge_attr.dim()==1:
            edge_attr=edge_attr.view(-1,1)
        edge_attr=edge_attr.to(dtype=torch.float)
        score=self.att(edge_attr)
        score=score.squeeze(-1)
        softmax_score=scatter_softmax(score,edge_index[1],dim=0)
        self._edge_weight=softmax_score

        return self.propagate(edge_index,x=(x_user,x_movie))

    def message(self, x_j):
        """
        x_j: source node features for each edge
        self._edge_weight: attention score per edge
        """
        return x_j * self._edge_weight.view(-1,1)


In [15]:
class GNNEncoder(torch.nn.Module):
    
    def __init__(self,hidden_channels,output_channels):
        
        super().__init__()
        self.lin_dict=ModuleDict({
            'user':torch.nn.LazyLinear(hidden_channels),
            'movie':torch.nn.LazyLinear(hidden_channels)
        })

        self.norm_dict=ModuleDict({
        'user':BatchNorm1d(hidden_channels),
        'movie':BatchNorm1d(hidden_channels)
        })

        self.drop=Dropout(0.2)

        self.att_layer = AttAwareConv()  # Attention Layer
        self.conv1 = HeteroConv({
            ('user', 'rates', 'movie'): SAGEConv((-1, -1), hidden_channels),
            ('movie', 'rated_by', 'user'): SAGEConv((-1, -1), hidden_channels),
        }, aggr='mean')

        self.conv2 = HeteroConv({
            ('user', 'rates', 'movie'): SAGEConv((-1, -1), output_channels),
            ('movie', 'rated_by', 'user'): SAGEConv((-1, -1), output_channels),
        }, aggr='mean')

        
    def forward(self, x_dict, edge_index_dict, edge_attr_dict):
        # Step 1: preprocess input features
        x_dict = {key: self.lin_dict[key](x) for key, x in x_dict.items()}
        x_dict = {key: self.norm_dict[key](x) for key, x in x_dict.items()}
        x_dict = {key: self.drop(F.relu(x)) for key, x in x_dict.items()}

        # Step 2: apply attention layer to user→movie edges
        edge_index = edge_index_dict[('user','rates','movie')]
        edge_attr = edge_attr_dict[('user','rates','movie')]
        #print("type:",type(edge_attr))
        att_out = self.att_layer(x_dict['user'], x_dict['movie'], edge_index, edge_attr)
        
        # Update movie features with attention-weighted user messages
        x_dict['movie'] = x_dict['movie'] + att_out

        # Step 3: apply graph convolutions
        x_dict = self.conv1(x_dict, {('user','rates','movie'): edge_index,
                                 ('movie','rated_by','user'): edge_index[[1, 0]]})
        x_dict = {key: self.drop(F.relu(x)) for key, x in x_dict.items()}
        x_dict = self.conv2(x_dict, {('user','rates','movie'): edge_index,
                                 ('movie','rated_by','user'): edge_index[[1, 0]]})

        return x_dict
    

In [16]:
class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.mlp = torch.nn.Sequential(
            Linear(2 * hidden_channels, 4 * hidden_channels),
            torch.nn.ReLU(),
            Dropout(0.2),
            Linear(4 * hidden_channels, 2 * hidden_channels),
            torch.nn.ReLU(),
            Linear(2 * hidden_channels, 1)
        )

    def forward(self, z_dict, edge_label_index):
        user_z = z_dict['user'][edge_label_index[0]]
        movie_z = z_dict['movie'][edge_label_index[1]]
        features = torch.cat([user_z, movie_z], dim=-1)
        return self.mlp(features).squeeze(-1)

In [17]:
class GNNRecommendationModel(torch.nn.Module):
    def __init__(self, hidden_channels, metadata):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index, edge_attr_dict):
        z_dict = self.encoder(x_dict, edge_index_dict, edge_attr_dict)
        return self.decoder(z_dict, edge_label_index)


In [18]:
model=GNNRecommendationModel(hidden_channels=64,metadata=train_graph.metadata()).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
criterion = torch.nn.MSELoss()

In [19]:
from torch_geometric.loader import LinkNeighborLoader

def prepare_loaders(train_graph, test_graph, batch_size=256):
    
    train_loader = LinkNeighborLoader(
        data=train_graph,
        num_neighbors=[10, 5],
        edge_label_index=(('user', 'rates', 'movie'), train_graph[('user', 'rates', 'movie')].edge_index),
        edge_label=train_graph[('user', 'rates', 'movie')].edge_attr,  
        batch_size=batch_size,
        shuffle=True,
    )
  
    test_loader = LinkNeighborLoader(
        data=test_graph,
        num_neighbors=[10, 5],
        edge_label_index=(('user', 'rates', 'movie'), test_graph[('user', 'rates', 'movie')].edge_index),
        edge_label=test_graph[('user', 'rates', 'movie')].edge_attr,
        batch_size=batch_size,
        shuffle=False
    )
    
    return train_loader, test_loader


In [20]:
train_loader, test_loader = prepare_loaders(train_graph, test_graph, batch_size=256)

In [ ]:
def train_epoch():
    model.train(); total=0
    for batch in train_loader:
        batch = batch.to(device)
        ea = {etype: batch[etype].edge_attr
              for etype in batch.edge_types if 'edge_attr' in batch[etype]}
        pred = model(batch.x_dict, batch.edge_index_dict,
                     batch[('user','rates','movie')].edge_label_index, ea)
        loss = F.mse_loss(pred, batch[('user','rates','movie')].edge_label)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        total += loss.item()*pred.size(0)
    return total/len(train_loader.dataset)

def eval_epoch(loader):
    model.eval(); tot=0
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            ea = {etype: batch[etype].edge_attr
                  for etype in batch.edge_types if 'edge_attr' in batch[etype]}
            pred = model(batch.x_dict, batch.edge_index_dict,
                         batch[('user','rates','movie')].edge_label_index, ea)
            tot += F.mse_loss(pred, batch[('user','rates','movie')].edge_label,
                              reduction='sum').item()
    rmse = (tot/len(loader.dataset))**0.5
    return rmse

for epoch in range(1, 5):
    train_loss = train_epoch()
    test_metrics = eval_epoch(test_loader)
    
    scheduler.step()
    
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, '
          f'Test RMSE: {test_metrics:.4f}')
    

torch.save(model.state_dict(), 'movie_recommender.pth')

Epoch: 001, Train Loss: 1.1069, Test RMSE: 1.0521
Epoch: 002, Train Loss: 1.1069, Test RMSE: 1.0521
Epoch: 003, Train Loss: 1.1069, Test RMSE: 1.0521


In [ ]:

def generate_recommendations(model, data, user_id, top_k=10, device='cpu'):
   
    model.eval()
    data = data.to(device)
    
    with torch.no_grad():
        
        z_dict = model.encoder(data.x_dict, data.edge_index_dict)
        

        if isinstance(user_id, int):
            user_id = torch.tensor([user_id], device=device)
        
        user_emb = z_dict['user'][user_id]  # shape: [1, emb_dim]
        movie_emb = z_dict['movie']         # shape: [num_movies, emb_dim]
        
        edge_label_index = torch.stack([
            user_id.repeat(movie_emb.size(0)),
            torch.arange(movie_emb.size(0), device=device)
        ])
        scores = model.decoder(z_dict, edge_label_index)  # shape: [num_movies]
        
        rated_mask = torch.zeros(movie_emb.size(0), dtype=torch.bool, device=device)
        user_edges = (data['user', 'rates', 'movie'].edge_index[0] == user_id)
        rated_movies = data['user', 'rates', 'movie'].edge_index[1][user_edges]
        rated_mask[rated_movies] = True
        scores[rated_mask] = -float('inf')
        
        top_scores, top_movies = torch.topk(scores, k=min(top_k, len(scores)))
        
        return top_movies.cpu().numpy(), top_scores.cpu().numpy()


id_list,score_list = generate_recommendations(model, train_graph, 50,device=device)

movie_titles = movies['title'].to_dict()


for i in range(10):
    original_id = id_list[i]
    print(f"{i+1}. {movie_titles.get(original_id)}  score:{score_list[i]:.5f}")
